# [초급 프로젝트] 4팀_김명환

---
---

# 프로그래밍

In [33]:
# 기본 라이브러리 (중복 제거 및 정리)

# --- Scikit-learn: 데이터 전처리, 모델, 평가 ---
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.datasets import (
    fetch_california_housing, load_iris, make_moons, make_circles,
    load_breast_cancer, load_wine
)
from sklearn import datasets
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier, plot_tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, mean_squared_error, average_precision_score

# --- 이미지 처리 ---
import cv2
from PIL import Image, ImageFilter, ImageDraw
import albumentations as A

# --- PyTorch: 딥러닝 관련 ---
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader, Subset
# 문제 있는 v2 import 제거하고 필요시에만 개별적으로 import
# from torchvision.transforms import v2, functional as TF
from torchvision.transforms import functional as TF
from torchvision.datasets import CocoDetection
from torch.nn import CrossEntropyLoss
from collections import OrderedDict

# --- COCO 데이터셋 관련 ---
from pycocotools.coco import COCO
from pycocotools import mask as coco_mask

# --- 딥러닝 모델 ---
import timm

# --- 기본 라이브러리 ---
import os
import sys
import re
import csv
import copy
import json
import math
import random
import yaml
import shutil
import requests
import xml.etree.ElementTree as ET
from pathlib import Path

# --- 데이터 분석 및 시각화 ---
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches

# --- 시간 관련 ---
from datetime import datetime, timezone, timedelta
import pytz

# --- 진행률 표시 ---
import IPython.display
from tqdm.notebook import tqdm

# --- 시간대 설정 ---
__kst = pytz.timezone('Asia/Seoul')

# --- GPU 설정 ---
__device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
__device_cpu = torch.device('cpu')

# --- 재현 가능한 결과를 위한 시드 설정 ---
np.random.seed(42)
torch.manual_seed(42)
if __device.type == 'cuda':
    torch.cuda.manual_seed_all(42)

print(f"라이브러리 로드 완료 사용장치: {__device}")

라이브러리 로드 완료 사용장치: cpu


In [34]:
from urllib.request import urlretrieve; urlretrieve("https://raw.githubusercontent.com/c0z0c/jupyter_hangul/refs/heads/beta/helper_c0z0c_dev.py", "helper_c0z0c_dev.py")
import importlib
import helper_c0z0c_dev as helper
importlib.reload(helper)

🌐 https://c0z0c.github.io/jupyter_hangul
ℹ️ NumPy 2.1.3 (v2.x+): 호환성 모드 적용됨
✅ 설정 완료: 한글 폰트, plt 전역 등록, pandas 확장, 캐시 기능
pd commit 저장 경로 = d:\GoogleDrive\codeit_ai_health_eat\scripts\김명환


<module 'helper_c0z0c_dev' from 'd:\\GoogleDrive\\codeit_ai_health_eat\\scripts\\김명환\\helper_c0z0c_dev.py'>

In [35]:
import os, sys
from pathlib import Path

utils_dir = None
if helper.is_colab:
    utils_dir = "/content/drive/MyDrive/codeit_ai_health_eat/src/python_modules/utils"
else:
    utils_dir = os.path.join(Path.cwd().drive + '\\', 'GoogleDrive', "codeit_ai_health_eat", "src", "python_modules", "utils")

print("utils_dir:", utils_dir)

sys.path.append(str(utils_dir))
print("sys.path:", sys.path)
import importlib
import health_ea_utils as heu
importlib.reload(heu)
from health_ea_utils import *

print("helper.__file__:", helper.__file__)
print("health_ea_utils.__file__:", heu.__file__)


utils_dir: d:\GoogleDrive\codeit_ai_health_eat\src\python_modules\utils
sys.path: ['c:\\Users\\sw1\\anaconda3\\envs\\env_colab_250827\\python310.zip', 'c:\\Users\\sw1\\anaconda3\\envs\\env_colab_250827\\DLLs', 'c:\\Users\\sw1\\anaconda3\\envs\\env_colab_250827\\lib', 'c:\\Users\\sw1\\anaconda3\\envs\\env_colab_250827', '', 'C:\\Users\\sw1\\AppData\\Roaming\\Python\\Python310\\site-packages', 'c:\\Users\\sw1\\anaconda3\\envs\\env_colab_250827\\lib\\site-packages', 'c:\\Users\\sw1\\anaconda3\\envs\\env_colab_250827\\lib\\site-packages\\win32', 'c:\\Users\\sw1\\anaconda3\\envs\\env_colab_250827\\lib\\site-packages\\win32\\lib', 'c:\\Users\\sw1\\anaconda3\\envs\\env_colab_250827\\lib\\site-packages\\Pythonwin', 'd:\\GoogleDrive\\codeit_ai_health_eat\\src\\python_modules\\utils', 'd:\\GoogleDrive\\codeit_ai_health_eat\\src\\python_modules\\utils', 'd:\\GoogleDrive\\codeit_ai_health_eat\\src\\python_modules\\utils', 'd:\\GoogleDrive\\codeit_ai_health_eat\\src\\python_modules\\utils', 'd:\\Go

# 1. 학습용 데이타 다운로드 및 압축 풀기

In [36]:
def get_tqdm_kwargs():
    """Widget 오류를 방지하는 안전한 tqdm 설정"""
    return {
        'disable': False,
        'leave': True,
        'file': sys.stdout,
        'ascii': True,  # ASCII 문자만 사용
        'dynamic_ncols': False,
#        'ncols': 80  # 고정 폭
    }

def drive_root():
    root_path = os.path.join("D:\\", "GoogleDrive")
    if helper.is_colab:
        root_path = os.path.join("/content/drive/MyDrive")
    return root_path

def get_path_modeling(add_path = None):
    modeling_path = "modeling_yolo"
    path = os.path.join(drive_root(),modeling_path)
    if add_path is not None:
        path = os.path.join(path,add_path)
    return path

def get_path_modeling_release(add_path = None):
    modeling_path = "modeling_yolo"
    path = os.path.join(drive_root(),modeling_path)
    if add_path is not None:
        path = os.path.join(path,add_path)
    return path

def print_dir_tree(root, max_depth=2, list_count=3, indent=""):
    import os
    if max_depth < 0:
        return
    try:
        items = os.listdir(root)
    except Exception as e:
        print(indent + f"[Error] {e}")
        return

    img_count = len([f for f in os.listdir(root) if f.lower().endswith(('.jpg', '.jpeg', '.png', '.xml', '.inf', '.txt'))])
    for item in items:
        path = os.path.join(root, item)
        if os.path.isdir(path):
            print(indent + "|-- "+ item)
            # 이미지 파일 개수만 출력
            img_count = len([f for f in os.listdir(path) if f.lower().endswith(('.jpg', '.jpeg', '.png', '.xml', '.inf', '.txt'))])
            if img_count > list_count:
                print(indent + "   "+ f"[데이터파일: {img_count}개]")
            print_dir_tree(root=path, max_depth=max_depth-1, list_count=list_count, indent=indent + "   ")
        else:
            if list_count < img_count and item.lower().endswith(('.jpg', '.jpeg', '.png', '.xml', '.inf', '.txt')):
                continue
            print(indent + "|-- "+ item)

def save_model_dict(model, path, pth_name, kwargs=None):
    """모델 state_dict와 추가 정보를 저장"""
    def safe_makedirs(path):
        """안전한 디렉토리 생성"""
        if os.path.exists(path) and not os.path.isdir(path):
            os.remove(path)  # 파일이면 삭제
        os.makedirs(path, exist_ok=True)

    # 디렉토리 생성
    safe_makedirs(path)

    # 모델 구조 정보 추출
    model_info = {
        'class_name': model.__class__.__name__,
        'init_args': {},
        'str': str(model),
        'repr': repr(model),
        'modules': [m.__class__.__name__ for m in model.modules()],
    }

    # 생성자 인자 자동 추출(가능한 경우)
    if hasattr(model, '__dict__'):
        for key in ['in_ch', 'base_ch', 'num_classes', 'out_ch']:
            if hasattr(model, key):
                model_info['init_args'][key] = getattr(model, key)

    # kwargs 처리
    extra_info = {}
    if kwargs is not None:
        if isinstance(kwargs, str):
            extra_info = json.loads(kwargs)
        elif isinstance(kwargs, dict):
            extra_info = kwargs

    model_info.update(extra_info)

    # 저장할 dict 구성
    save_dict = {
        'model_state': model.state_dict(),
        'class_name': model.__class__.__name__,
        'model_info': model_info,
    }

    save_path = os.path.join(path, f"{pth_name}.pth")
    torch.save(save_dict, save_path)
    return save_path

def load_model_dict(path, pth_name=None):
    """
    save_model_dict로 저장한 모델을 불러오는 함수
    반환값: (model_state, model_info)
    """
    import torch
    load_path = path
    if pth_name is not None:
        load_path = os.path.join(path, f"{pth_name}.pth")
    checkpoint = torch.load(load_path, map_location='cpu', weights_only=False)  # <-- 여기 추가
    model_state = checkpoint.get('model_state')
    model_info = checkpoint.get('model_info')
    model_info['file_name'] = os.path.basename(load_path)
    return model_state, model_info


def search_pth_files(base_path):
    """
    입력된 경로의 하위 폴더들에서 pth 파일들을 검색
    """
    pth_files = []

    if not os.path.exists(base_path):
        print(f"경로가 존재하지 않습니다: {base_path}")
        return pth_files

    print(f"pth 파일 검색 시작: {base_path}")

    # 하위 폴더들을 순회하며 pth 파일 검색
    for root, dirs, files in os.walk(base_path):
        for file in files:
            if file.endswith('.pth'):
                pth_path = os.path.join(root, file)
                pth_files.append(pth_path)

    # 결과 정리 및 출력
    if pth_files:
        print(f"\n발견된 pth 파일들 ({len(pth_files)}개):")
        for i, pth_file in enumerate(pth_files, 1):
            # 상대 경로로 표시 (base_path 기준)
            rel_path = os.path.relpath(pth_file, base_path)
            print(f" {i:2d}. {rel_path}")
    else:
        print("pth 파일을 찾을 수 없습니다.")

    return pth_files

def print_json_tree(data, indent="", max_depth=4, _depth=0, list_count=2, print_value=True):
    """
    JSON 객체를 지정한 단계(max_depth)까지 트리 형태로 출력
    - list 타입은 3개 이상일 때 개수만 출력
    - 하위 노드가 값일 경우 key(type) 형태로 출력
    - print_value=True일 때 key(type): 값 형태로 출력
    """
    if _depth > max_depth:
        return
    if isinstance(data, dict):
        for key, value in data.items():
            if isinstance(value, (dict, list)):
                print(f"{indent}|-- {key}")
                print_json_tree(value, indent + "    ", max_depth, _depth + 1, list_count, print_value)
            else:
                if print_value:
                    print(f"{indent}|-- {key}({type(value).__name__}): {value if len(str(value)) < 100 else f'{str(value)[:30]}...'}")
                else:
                    print(f"{indent}|-- {key}({type(value).__name__})")
    elif isinstance(data, list):
        if len(data) > list_count:
            print(f"{indent}|-- [list] ({len(data)} items)")
        else:
            for i, item in enumerate(data):
                if isinstance(item, (dict, list)):
                    print(f"{indent}|-- [{i}]")
                    print_json_tree(item, indent + "    ", max_depth, _depth + 1, list_count, print_value)
                else:
                    if print_value:
                        print(f"{indent}|-- [{i}]({type(item).__name__}): {item if len(str(item)) < 100 else f'{str(item)[:30]}...'}")
                    else:
                        print(f"{indent}|-- [{i}]({type(item).__name__})")
    else:
        if print_value:
            print(f"{indent}{type(data).__name__}: {data if len(str(data)) < 100 else f'{str(data)[:30]}...'}")
        else:
            print(f"{indent}{type(data).__name__}")

def print_git_tree(data, indent="", max_depth=3, _depth=0):
    """
    PyTorch tensor/딕셔너리/리스트를 git tree 스타일로 출력
    """
    import torch
    import numpy as np

    if _depth > max_depth:
        return
    if isinstance(data, dict):
        for key, value in data.items():
            print(f"{indent}├─ {key} [{type(value).__name__}]")
            print_git_tree(value, indent + "│  ", max_depth, _depth + 1)
    elif isinstance(data, (list, tuple)):
        for i, item in enumerate(data):
            print(f"{indent}├─ [{i}] [{type(item).__name__}]")
            print_git_tree(item, indent + "│  ", max_depth, _depth + 1)
    elif torch.is_tensor(data):
        shape = tuple(data.shape)
        dtype = str(data.dtype)
        preview = str(data)
        preview_str = preview[:80] + ("..." if len(preview) > 80 else "")
        print(f"{indent}└─ Tensor shape={shape} dtype={dtype} preview={preview_str}")
    elif isinstance(data, np.ndarray):
        shape = data.shape
        dtype = data.dtype
        preview = str(data)
        preview_str = preview[:80] + ("..." if len(preview) > 80 else "")
        print(f"{indent}└─ ndarray shape={shape} dtype={dtype} preview={preview_str}")
    else:
        val_str = str(data)
        print(f"{indent}└─ {type(data).__name__}: {val_str[:80]}{'...' if len(val_str)>80 else ''}")


print("유틸리티 함수 로드 완료")

유틸리티 함수 로드 완료


In [37]:
import os

# 전체 프로젝트 디렉토리에서 pkl 파일 검색
base_dir = os.path.join(drive_root(), "codeit_ai_health_eat")
print(f"검색 시작 디렉토리: {base_dir}")

pkl_files = []
for root, dirs, files in os.walk(base_dir):
    for file in files:
        if file.endswith('.pkl'):
            pkl_path = os.path.join(root, file)
            pkl_files.append(pkl_path)
            
if pkl_files:
    print(f"\n발견된 pkl 파일들 ({len(pkl_files)}개):")
    for i, pkl_file in enumerate(pkl_files, 1):
        rel_path = os.path.relpath(pkl_file, base_dir)
        print(f" {i:2d}. {rel_path}")
else:
    print("pkl 파일을 찾을 수 없습니다.")

검색 시작 디렉토리: D:\GoogleDrive\codeit_ai_health_eat

발견된 pkl 파일들 (2개):
  1. src\python_modules\data\df_drug.pkl
  2. src\python_modules\data\df_병용금기약물_20240813.pkl


In [38]:
df_drug = pd.read_pickle(os.path.join(drive_root(), "codeit_ai_health_eat", 
                                      "src", 
                                      "python_modules", 
                                      "data", 
                                      "df_class.pkl"))


FileNotFoundError: [Errno 2] No such file or directory: 'D:\\GoogleDrive\\codeit_ai_health_eat\\src\\python_modules\\data\\df_class.pkl'

In [ ]:
df_drug.head_att(10)

     drug_N category_id   drug_S                          dl_name                                dl_name_en                                                     img_key           dl_material                  dl_material_en dl_custom_shape         dl_company               dl_company_en                            di_class_no di_etc_otc_code         di_edi_code                                       chart drug_shape   form_code_name supercategory                             name class_id
 0 K-001900        1899 정상알약                 보령부스파정 5mg                   Buspar Tab. 5mg Boryung        http://connectdi.com/design/img/drug/1Mxwka5v0lL.jpg        부스피론염산염         Buspirone Hydrochloride    정제, 저작정       보령제약(주)                     Boryung                    [01170]정신신경용제      전문의약품 641901280,A09302381     이약은 양면볼록한 장방형의 흰색정제이다     장방형             나정          pill                 보령부스파정 5mg        1
 1 K-002483        2482 정상알약                 뮤테란캡슐 100mg                        Muteran Cap

In [ ]:

df = pd.read_pickle(os.path.join(drive_root(), "codeit_ai_health_eat", 
                                      "src", 
                                      "python_modules", 
                                      "data", 
                                      "df_병용금기약물_20240813.pkl"))

In [ ]:
# D:\temp\한국의약품안전관리원_병용금기약물_20240813\한국의약품안전관리원_병용금기약물_20240813.csv
# df = pd.read_csv(r"D:\temp\한국의약품안전관리원_병용금기약물_20240813\한국의약품안전관리원_병용금기약물_20240813.csv", encoding="utf-8")
# df.to_pickle("한국의약품안전관리원_병용금기약물_20240813.pkl")

In [ ]:
# df에서 고유한 코드(값)가 있는 필드(컬럼) 찾기
unique_code_fields = []

for col in df.columns:
    # 컬럼의 값 개수와 고유값 개수 비교 (고유값이 전체 row와 같으면 코드 필드로 간주)
    if df[col].nunique() == len(df):
        unique_code_fields.append(col)

print("고유한 코드(값)가 있는 필드:", unique_code_fields)

고유한 코드(값)가 있는 필드: []


In [47]:
df_drug.head_att(10)

     drug_N category_id   drug_S                          dl_name                                dl_name_en                                                     img_key           dl_material                  dl_material_en dl_custom_shape         dl_company               dl_company_en                            di_class_no di_etc_otc_code         di_edi_code                                       chart drug_shape   form_code_name supercategory                             name class_id
 0 K-001900        1899 정상알약                 보령부스파정 5mg                   Buspar Tab. 5mg Boryung        http://connectdi.com/design/img/drug/1Mxwka5v0lL.jpg        부스피론염산염         Buspirone Hydrochloride    정제, 저작정       보령제약(주)                     Boryung                    [01170]정신신경용제      전문의약품 641901280,A09302381     이약은 양면볼록한 장방형의 흰색정제이다     장방형             나정          pill                 보령부스파정 5mg        1
 1 K-002483        2482 정상알약                 뮤테란캡슐 100mg                        Muteran Cap

In [52]:
df.describe().head_att(10)
print(df.dtypes)

            제품코드A      제품코드B      공고번호
 count         542996         542996        542996
  mean 607167331.0012 602228294.9372 20145407.8066
   std 155780604.9291 162994388.8566    64507.3029
   min       51600330       51600171       2021113
   25%      642703980      642200630      20110110
   50%      647302710      648104800      20150021
   75%      657308460      657200921      20200061
   max      699917050      698504790      20240030
성분명A     object
성분코드A    object
제품코드A     int64
제품명A     object
업체명A     object
급여구분A    object
성분명B     object
성분코드B    object
제품코드B     int64
제품명B     object
업체명B     object
급여구분B    object
공고번호      int64
공고일자     object
금기사유     object
dtype: object


In [50]:
df[df['제품코드B'] == 641901280].head_att(10)
# 1900

                         성분명A 성분코드A 제품코드A                            제품명A          업체명A 급여구분A                 성분명B 성분코드B 제품코드B                                         제품명B  업체명B 급여구분B 공고번호   공고일자 금기사유
 381955 selegiline hydrochloride 226401ATB 670700270 마오비정(셀레길린염산염)_(5mg/1정) 초당약품공업(주)      급여 buspirone hydrochloride 120502ATB 641901280 보령부스파정5밀리그램(부스피론염산염)_(5mg/1정) (주)보령      급여 20040002 2004-01-16 혈압상승


In [46]:
dfa = df['제품명B'].astype(str).str.contains('보령부스파정', na=False)
df[dfa].head_att(10)

                         성분명A 성분코드A 제품코드A                            제품명A          업체명A 급여구분A                 성분명B 성분코드B 제품코드B                                           제품명B  업체명B 급여구분B 공고번호   공고일자 금기사유
 381666 selegiline hydrochloride 226401ATB 670700270 마오비정(셀레길린염산염)_(5mg/1정) 초당약품공업(주)      급여 buspirone hydrochloride 120501ATB 641901270 보령부스파정10밀리그램(부스피론염산염)_(10mg/1정) (주)보령      급여 20040002 2004-01-16 혈압상승
 381955 selegiline hydrochloride 226401ATB 670700270 마오비정(셀레길린염산염)_(5mg/1정) 초당약품공업(주)      급여 buspirone hydrochloride 120502ATB 641901280   보령부스파정5밀리그램(부스피론염산염)_(5mg/1정) (주)보령      급여 20040002 2004-01-16 혈압상승
 381979 selegiline hydrochloride 226401ATB 670700270 마오비정(셀레길린염산염)_(5mg/1정) 초당약품공업(주)      급여 buspirone hydrochloride 120503ATB 641905340       보령부스파정15mg(부스피론염산염)_(15mg/1정) (주)보령      급여 20040002 2004-01-16 혈압상승


In [ ]:
def find_contraindicated_combinations(df, korean_ingredient_list):
    """
    한글 성분명 리스트를 받아서 병용금기 약물 조합을 찾는 함수
    
    Args:
        df: 병용금기 데이터프레임
        korean_ingredient_list: 사진에서 인식된 한글 성분명 리스트
    
    Returns:
        병용금기 조합 정보가 담긴 리스트
    """
    results = []
    
    # 각 성분에 대해 병용금기 정보 검색
    for ingredient in korean_ingredient_list:
        # 제품명A 또는 제품명B에 해당 성분이 포함된 행들 찾기
        mask_a = df['제품명A'].astype(str).str.contains(ingredient, na=False)
        mask_b = df['제품명B'].astype(str).str.contains(ingredient, na=False)
        
        # 해당 성분이 포함된 모든 병용금기 정보
        matching_rows = df[mask_a | mask_b]
        
        for _, row in matching_rows.iterrows():
            # 검색된 성분이 A에 있는지 B에 있는지 확인
            if ingredient in str(row['제품명A']):
                contraindicated_with = row['제품명B']
                contraindicated_ingredient = row['성분명B']
                current_ingredient = row['성분명A']
            else:
                contraindicated_with = row['제품명A']
                contraindicated_ingredient = row['성분명A']
                current_ingredient = row['성분명B']
            
            result_info = {
                '검색_성분': ingredient,
                '현재_성분명': current_ingredient,
                '금기_제품명': contraindicated_with,
                '금기_성분명': contraindicated_ingredient,
                '금기사유': row['금기사유'],
                '공고일자': row['공고일자']
            }
            results.append(result_info)
    
    return results

def check_mutual_contraindications(df, korean_ingredient_list):
    """
    입력된 성분들 간의 상호 병용금기 여부를 체크하는 함수
    
    Args:
        df: 병용금기 데이터프레임
        korean_ingredient_list: 사진에서 인식된 한글 성분명 리스트
    
    Returns:
        상호 병용금기 조합 정보
    """
    dangerous_combinations = []
    
    # 각 성분 쌍에 대해 병용금기 여부 확인
    for i in range(len(korean_ingredient_list)):
        for j in range(i+1, len(korean_ingredient_list)):
            ingredient_a = korean_ingredient_list[i]
            ingredient_b = korean_ingredient_list[j]
            
            # A에 ingredient_a가 있고 B에 ingredient_b가 있는 경우
            mask1 = (df['제품명A'].astype(str).str.contains(ingredient_a, na=False) & 
                    df['제품명B'].astype(str).str.contains(ingredient_b, na=False))
            
            # A에 ingredient_b가 있고 B에 ingredient_a가 있는 경우
            mask2 = (df['제품명A'].astype(str).str.contains(ingredient_b, na=False) & 
                    df['제품명B'].astype(str).str.contains(ingredient_a, na=False))
            
            contraindicated_rows = df[mask1 | mask2]
            
            if not contraindicated_rows.empty:
                for _, row in contraindicated_rows.iterrows():
                    danger_info = {
                        '성분_조합': f"{ingredient_a} + {ingredient_b}",
                        '제품명A': row['제품명A'],
                        '제품명B': row['제품명B'],
                        '금기사유': row['금기사유'],
                        '공고일자': row['공고일자'],
                        '위험도': '높음'
                    }
                    dangerous_combinations.append(danger_info)
    
    return dangerous_combinations

def display_contraindication_results(results, mutual_results):
    """
    병용금기 검색 결과를 보기 좋게 출력하는 함수
    """
    print("=" * 80)
    print("🚨 병용금기 약물 검색 결과")
    print("=" * 80)
    
    if mutual_results:
        print("\n⚠️  위험! 입력된 성분들 간의 직접적인 병용금기 발견:")
        print("-" * 60)
        for result in mutual_results:
            print(f"• {result['성분_조합']}")
            print(f"  금기사유: {result['금기사유']}")
            print(f"  공고일자: {result['공고일자']}")
            print()
    
    if results:
        print(f"\n📋 각 성분별 병용금기 약물 정보 ({len(results)}건):")
        print("-" * 60)
        
        # 성분별로 그룹화
        from collections import defaultdict
        grouped = defaultdict(list)
        for result in results:
            grouped[result['검색_성분']].append(result)
        
        for ingredient, ingredient_results in grouped.items():
            print(f"\n🔍 '{ingredient}' 관련 병용금기:")
            for result in ingredient_results[:5]:  # 상위 5개만 표시
                print(f"  ❌ {result['금기_제품명']} ({result['금기_성분명']})")
                print(f"     사유: {result['금기사유'][:50]}{'...' if len(result['금기사유']) > 50 else ''}")
            if len(ingredient_results) > 5:
                print(f"  ... 외 {len(ingredient_results) - 5}건 더")
    
    if not results and not mutual_results:
        print("\n✅ 입력된 성분들에 대한 병용금기 정보가 발견되지 않았습니다.")

# 사용 예시
korean_ingredient_list = ["비칼루타마이드", "블로난세린", "돔페리돈말레산염"]

# 각 성분별 병용금기 정보 검색
contraindication_results = find_contraindicated_combinations(df, korean_ingredient_list)

# 입력된 성분들 간의 상호 병용금기 체크
mutual_contraindications = check_mutual_contraindications(df, korean_ingredient_list)

# 결과 출력
display_contraindication_results(contraindication_results, mutual_contraindications)

🚨 병용금기 약물 검색 결과

⚠️  위험! 입력된 성분들 간의 직접적인 병용금기 발견:
------------------------------------------------------------
• 비칼루타마이드 + 돔페리돈말레산염
  금기사유: QTc 연장 효과 증대로 심각한 위험(Torsade de Pointes, 심각한 심실성 부정맥 포함) 가능성
  공고일자: 2015-11-30

• 비칼루타마이드 + 돔페리돈말레산염
  금기사유: QTc 연장 효과 증대로 심각한 위험(Torsade de Pointes, 심각한 심실성 부정맥 포함) 가능성
  공고일자: 2015-11-30

• 비칼루타마이드 + 돔페리돈말레산염
  금기사유: QTc 연장 효과 증대로 심각한 위험(Torsade de Pointes, 심각한 심실성 부정맥 포함) 가능성
  공고일자: 2015-11-30

• 비칼루타마이드 + 돔페리돈말레산염
  금기사유: QTc 연장 효과 증대로 심각한 위험(Torsade de Pointes, 심각한 심실성 부정맥 포함) 가능성
  공고일자: 2015-11-30

• 비칼루타마이드 + 돔페리돈말레산염
  금기사유: QTc 연장 효과 증대로 심각한 위험(Torsade de Pointes, 심각한 심실성 부정맥 포함) 가능성
  공고일자: 2015-11-30

• 비칼루타마이드 + 돔페리돈말레산염
  금기사유: QTc 연장 효과 증대로 심각한 위험(Torsade de Pointes, 심각한 심실성 부정맥 포함) 가능성
  공고일자: 2015-11-30

• 비칼루타마이드 + 돔페리돈말레산염
  금기사유: QTc 연장 효과 증대로 심각한 위험(Torsade de Pointes, 심각한 심실성 부정맥 포함) 가능성
  공고일자: 2015-11-30

• 비칼루타마이드 + 돔페리돈말레산염
  금기사유: QTc 연장 효과 증대로 심각한 위험(Torsade de Pointes, 심각한 심실성 부정맥 포함) 가능성
  공고일자: 2015-11-30

• 비칼루타마이드

In [ ]:
def find_contraindicated_combinations(df, korean_ingredient_list):
    """
    한글 성분명 리스트를 받아서 병용금기 약물 조합을 찾는 함수
    
    Args:
        df: 병용금기 데이터프레임
        korean_ingredient_list: 사진에서 인식된 한글 성분명 리스트
    
    Returns:
        병용금기 조합 정보가 담긴 리스트 (중복 제거됨)
    """
    results = []
    seen_combinations = set()  # 중복 제거를 위한 set
    
    # 각 성분에 대해 병용금기 정보 검색
    for ingredient in korean_ingredient_list:
        # 제품명A 또는 제품명B에 해당 성분이 포함된 행들 찾기
        mask_a = df['제품명A'].astype(str).str.contains(ingredient, na=False)
        mask_b = df['제품명B'].astype(str).str.contains(ingredient, na=False)
        
        # 해당 성분이 포함된 모든 병용금기 정보
        matching_rows = df[mask_a | mask_b]
        
        for _, row in matching_rows.iterrows():
            # 검색된 성분이 A에 있는지 B에 있는지 확인
            if ingredient in str(row['제품명A']):
                contraindicated_with = row['제품명B']
                contraindicated_ingredient = row['성분명B']
                current_ingredient = row['성분명A']
            else:
                contraindicated_with = row['제품명A']
                contraindicated_ingredient = row['성분명A']
                current_ingredient = row['성분명B']
            
            # 중복 제거를 위한 고유 키 생성 (성분명 기준)
            combo_key = f"{ingredient}|{contraindicated_ingredient}|{row['금기사유']}"
            
            if combo_key not in seen_combinations:
                seen_combinations.add(combo_key)
                result_info = {
                    '검색_성분': ingredient,
                    '현재_성분명': current_ingredient,
                    '금기_제품명': contraindicated_with,
                    '금기_성분명': contraindicated_ingredient,
                    '금기사유': row['금기사유'],
                    '공고일자': row['공고일자']
                }
                results.append(result_info)
    
    return results

def check_mutual_contraindications(df, korean_ingredient_list):
    """
    입력된 성분들 간의 상호 병용금기 여부를 체크하는 함수
    
    Args:
        df: 병용금기 데이터프레임
        korean_ingredient_list: 사진에서 인식된 한글 성분명 리스트
    
    Returns:
        상호 병용금기 조합 정보 (중복 제거됨)
    """
    dangerous_combinations = []
    seen_mutual_combinations = set()  # 중복 제거를 위한 set
    
    # 각 성분 쌍에 대해 병용금기 여부 확인
    for i in range(len(korean_ingredient_list)):
        for j in range(i+1, len(korean_ingredient_list)):
            ingredient_a = korean_ingredient_list[i]
            ingredient_b = korean_ingredient_list[j]
            
            # A에 ingredient_a가 있고 B에 ingredient_b가 있는 경우
            mask1 = (df['제품명A'].astype(str).str.contains(ingredient_a, na=False) & 
                    df['제품명B'].astype(str).str.contains(ingredient_b, na=False))
            
            # A에 ingredient_b가 있고 B에 ingredient_a가 있는 경우
            mask2 = (df['제품명A'].astype(str).str.contains(ingredient_b, na=False) & 
                    df['제품명B'].astype(str).str.contains(ingredient_a, na=False))
            
            contraindicated_rows = df[mask1 | mask2]
            
            if not contraindicated_rows.empty:
                for _, row in contraindicated_rows.iterrows():
                    # 중복 제거를 위한 고유 키 생성
                    combo_key = f"{ingredient_a}+{ingredient_b}|{row['금기사유']}"
                    
                    if combo_key not in seen_mutual_combinations:
                        seen_mutual_combinations.add(combo_key)
                        danger_info = {
                            '성분_조합': f"{ingredient_a} + {ingredient_b}",
                            '제품명A': row['제품명A'],
                            '제품명B': row['제품명B'],
                            '금기사유': row['금기사유'],
                            '공고일자': row['공고일자'],
                            '위험도': '높음'
                        }
                        dangerous_combinations.append(danger_info)
    
    return dangerous_combinations

def display_contraindication_results(results, mutual_results):
    """
    병용금기 검색 결과를 보기 좋게 출력하는 함수
    """
    print("=" * 80)
    print("🚨 병용금기 약물 검색 결과")
    print("=" * 80)
    
    if mutual_results:
        print(f"\n⚠️  위험! 입력된 성분들 간의 직접적인 병용금기 발견 ({len(mutual_results)}건):")
        print("-" * 60)
        for result in mutual_results:
            print(f"• {result['성분_조합']}")
            print(f"  금기사유: {result['금기사유']}")
            print(f"  공고일자: {result['공고일자']}")
            print()
    
    if results:
        print(f"\n📋 각 성분별 병용금기 약물 정보 ({len(results)}건):")
        print("-" * 60)
        
        # 성분별로 그룹화
        from collections import defaultdict
        grouped = defaultdict(list)
        for result in results:
            grouped[result['검색_성분']].append(result)
        
        for ingredient, ingredient_results in grouped.items():
            print(f"\n🔍 '{ingredient}' 관련 병용금기 ({len(ingredient_results)}건):")
            for result in ingredient_results[:5]:  # 상위 5개만 표시
                #print(f"  ❌ {result['금기_제품명']} ({result['금기_성분명']})")
                print(f"  ❌ {result['금기_제품명']}")
                print(f"     사유: {result['금기사유'][:50]}{'...' if len(result['금기사유']) > 50 else ''}")
            if len(ingredient_results) > 5:
                print(f"  ... 외 {len(ingredient_results) - 5}건 더")
    
    if not results and not mutual_results:
        print("\n✅ 입력된 성분들에 대한 병용금기 정보가 발견되지 않았습니다.")

# 사용 예시
# korean_ingredient_list = ["비칼루타마이드", "블로난세린", "돔페리돈말레산염"]

korean_ingredient_list = ["부스피론염산염", "가바펜틴", "에스오메프라졸스트론튬사수화물"]


# 각 성분별 병용금기 정보 검색 (중복 제거됨)
contraindication_results = find_contraindicated_combinations(df, korean_ingredient_list)

# 입력된 성분들 간의 상호 병용금기 체크 (중복 제거됨)
mutual_contraindications = check_mutual_contraindications(df, korean_ingredient_list)

# 결과 출력
display_contraindication_results(contraindication_results, mutual_contraindications)

🚨 병용금기 약물 검색 결과

📋 각 성분별 병용금기 약물 정보 (4건):
------------------------------------------------------------

🔍 '부스피론염산염' 관련 병용금기 (1건):
  ❌ 마오비정(셀레길린염산염)_(5mg/1정)
     사유: 혈압상승

🔍 '에스오메프라졸스트론튬사수화물' 관련 병용금기 (3건):
  ❌ 에보타즈정_(1정)
     사유: 아타자나비르의 혈중농도 감소
  ❌ 레야타즈캡슐200밀리그램(아타자나비어황산염)_(0.22779g/1캡슐)
     사유: 아타자나비르의 혈중농도 감소
  ❌ 에듀란트정25밀리그램(릴피비린염산염)_(27.5mg/1정)
     사유: 위장 pH의 증가로 릴피비린의 흡수가 저하되어 릴피비린의 치료효과 감소
